# Histogram2D time transfer

In [1]:
import time
import numpy as np
import asyncio as aio
import pandas as pd
from ipytablewidgets import LZ4Compressor
from vega.widget import VegaWidget
import ipywidgets as widgets
nbins = 256
spec_no_data = {'$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json',
 "width": 500,
 "height": 400,
 'data': {'name': 'data'},
 'encoding': {'color': {'field': 'z', 
                        'type': 'quantitative',
                        # "scale": {
                        # "domain": [0,1]  # Remove if domain changes
                        # }
                        },
  'x': {'field': 'x', 'type': 'ordinal'},
  'y': {'field': 'y', 'type': 'ordinal'}},
 'mark': 'rect',
 "config": {
    "axis": {
      "disable": True  # Change to False to see the ticks
    }
  }}
start = 0
times = []
callback_ = None
steps = 20
cases = ['vega', 'itw', 'lz4', 'zlib']

progress = widgets.IntProgress(value=0, min=0, 
                                     max=steps, 
                                     description='Progress', 
                                     orientation='horizontal')              

mean_d = {k: widgets.Text(value="", description='', disables=True)
          for k in cases}
        
std_d = {k: widgets.Text(value="",  description='', disables=True)
         for k in cases}

_lab = widgets.Label

items = ([_lab('Format'), _lab('Vega'), _lab('Table'), _lab('Table+lz4'), _lab('Table+zlib')] + 
         [_lab('Mean time [transf / transf+conv / total]')]+list(mean_d.values())+
         [_lab('Std time [transf / transf+conv / total]')]+list(std_d.values()))

grid = widgets.GridBox(items, layout=widgets.Layout(grid_template_columns="250px repeat(4, 150px)"))

mean = None
std = None

def clear_():
    times.clear()
    progress.value = 0
    mean.value = ''
    std.value = ''

def set_col(k):
    global progress, mean, std
    #progress = progress_d[k]
    mean = mean_d[k]
    std = std_d[k]
    if progress is not None:
        clear_()
    
def on_value_change(change):
    global start
    transf, tr_conv, total = change['new']
    times.append([transf-start, tr_conv-start, total-start])
    progress.value = len(times)
    if callback_ is not None and len(times)<steps:
        callback_()
    else:
        times_tr = [e[0] for e in times]
        times_tc = [e[1] for e in times]
        times_tt = [e[2] for e in times]
        mean_tr = np.mean(times_tr, dtype=int)
        mean_tc = np.mean(times_tc, dtype=int)
        mean_tt = np.mean(times_tt, dtype=int)
        std_tr = np.std(times_tr, dtype=int)
        std_tc = np.std(times_tc, dtype=int)
        std_tt = np.std(times_tt, dtype=int)
        mean.value = f"{mean_tr} / {mean_tc} / {mean_tt} ms"
        std.value = f"{std_tr} / {std_tc} / {std_tt} ms"
        
widget = VegaWidget(spec=spec_no_data)
widget.observe(on_value_change, names='rec_time')
display(widgets.VBox([widget, progress, grid]))
now_ = int(time.time()*1000)
#widget.rec_time = [now_, now_, now_]


nsamples=100_000
means = [0.1, 0.3]
cov = [[0.01, 0], [0, 0.09]]
rdata = np.random.multivariate_normal(means, cov, size=(nsamples))
h, *_ = np.histogram2d(rdata[:,0],rdata[:,1], bins=nbins, range=((-1.0,1.0),(-1.0,1.0)))
hist = h.astype('uint32')
widget._displayed = True

## Update with vega formated data

In [4]:
set_col('vega')
def vega_fun():
    global start
    vega_data = []
    for i in range(hist.shape[0]):
        for j in range(hist.shape[1]):
            vega_data.append(dict(x=i, y=j, z=int(hist[i,j])))
    start = int(time.time()*1000)
    widget.update('data', insert=vega_data, remove="true")
    time.sleep(1)
callback_ = vega_fun
vega_fun()

## Update with ipytablewidgets

In [5]:
set_col('itw')
widget.compression = None
def itw_fun():
    global start
    start = int(time.time()*1000)
    widget.update('data', insert=hist, remove="true")
    time.sleep(1)
callback_ = itw_fun
itw_fun()

## Update with ipytablewidgets + LZ4 compression

In [6]:
set_col('lz4')
start = int(time.time()*1000)
widget.compression = "lz4"
widget.update('data', insert=hist, remove="true")

## Update with ipytablewidgets + ZLIB compression

In [7]:
set_col('zlib')
start = int(time.time()*1000)
widget.compression = "zlib"
widget.update('data', insert=hist, remove="true")